# Onderzoek Data Science: Films
----

Voor de casusopdracht van het vak Data Science moet er een pipeline worden gemaakt, waarbij een toegewezen dataset wordt geanalyseerd. Dit data onderzoek is gebaseerd op de dataset over films en is opgezet door groep 6 van klas V2C. De dataset is toegewezen door de Hogeschool Utrecht en bevat onder andere:
- filmgegevens, waaronder duur, genres, taal, land van herkomst, budget en opbrengst;
- likes op facebook voor regisseur, hoofdrolspelers, totale cast en de film zelf;
- score op IMDB en aantal reviews. 


#### Hoofdvraag
Voor dit onderzoek is er een verplichte onderzoeksvraag, die luidt als volgt: 
- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?

#### Deelvragen
Het bevat een aantal onderzoeksvragen die wij zullen behandelen:

|Beeordeling                    |Deelvraag                                                                             |
|:------------------------------|:-------------------                                                                  |
|Externe dataset                |Hoeveel effect heeft de lengte van een trailer op de omzet van de film?               |
|Interactieve visualisatie      |Wat is de totale winst van alle films per jaar?                                       |
|Correlatieonderzoek            |Is een film succesvoller op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op facebook?                                                                                              | 
|Supervised Machine Learning    |In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?                                                                                                          |
|Unsupervised Machine Learning  |Is een film succesvoller op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op facebook?                                                                                              |

#### Z-test
Voor de hypothesetoets Z-test moet het volgende gebeuren.	
Een filmcriticus stelt dat de score van Engelstalige films lager is dan gemiddeld.
Wij moeten onderzoeken met de dataset of deze filmcriticus gelijk heeft. We nemen een steekproef (met pandas.DataFrame.sample(n=100,random_state=1)) van 100 Engelstalige films en beschouwen de hele dataset als populatie. Ook nemen we als betrouwbaarheid 90%. We gebruiken van de dataset alleen de filmgegevens waarbij zowel de taal (language) als de score (imdb_score) bekend zijn.

#### Teamleden
Het team bestaat uit de volgende drie personen:
- Sebastiaan Jansen
- Skott de Koster
- Mustafa Toprak

### Inhoudsopgave
---
1. Data Collection
2. Data Processing
3. Data Cleaning
4. Data Exploration & Analysis
5. Model building
6. Visualization
7. Communication

## Stap 1: Data collection
---

De dataset is aan ons gegeven door de Hogeschool Utrecht als onderdeel van de opdracht. Deze dataset houd een groot CSV bestand in met informatie over films. We hebben dus nu de data verzameld en gaan het nu inlezen.
De dataset is opgehaald van de volgende GitHub link: https://github.com/tijmenjoppe/ComputationalModelling-student/tree/master/casus/movie

Ook maken wij gebruik van een externe dataset. Deze externe dataset bevat een link ID naar de trailers van de films die op YouTube te vinden zijn. De externe dataset is opgehaald van de link: https://grouplens.org/datasets/movielens/20m-youtube/

## Stap 2: Data processing
---

We gaan nu de dataset inlezen en verwerken om het beter te kunnen bekijken. Om te beginnen importeren we de benodigde Python libraries:

In [ ]:
# Deze libraries zijn voor het verwerken van de data.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

# Deze libraries zijn voor het verkrijgen van YouTube video gegevens. Dit is relevant voor het gebruik van de externe dataset.
import pafy
from youtube_dl import YoutubeDL
import datetime 

Vervolgens lezen we de eerste (toegewezen) dataset in. Hiervoor gebruiken we pandas.read_csv en slaan we dit op in de dataframe "films". 
Aangezien de dataset erg groot is, laten we voor nu alleen de eerste 5 rijen zien. Dit doen we door een variabele films.head() aan te roepen:

In [ ]:
films = pd.read_csv('movie.csv', sep=",")
films.head()

Hetzelfde doen we met de tweede (externe) dataset, deze noemen we "films_extern": 

In [ ]:
films_extern = pd.read_csv('ml-youtube.csv', sep=",")
films_extern.head()

We zien dus dat de eerste dataset uit veel films bestaat. De tweede dataset heeft er nog veel meer, maar niet alle data in de sets is bruikbaar, dus dat gaan we nu schoonmaken.

# Stap 3: Data cleaning
---

We gaan nu de data schoon maken. Om te beginnen schonen we beide datasets op door alle data die niet relevant is te verwijderen:

In [ ]:
to_drop = ['color','facenumber_in_poster','country','aspect_ratio','content_rating']
films.drop(to_drop, inplace=True, axis=1)

films_extern.drop('movieId', inplace=True, axis=1)

In [ ]:
# We verwijderen de rijen met NaN gegevens in beide datasets, aangezien dit data is die we niet kunnen gebruiken.
films = films.dropna()
films_extern = films_extern.dropna()

In [ ]:
# We halen alle duplicates eruit.
films = films.drop_duplicates(subset="movie_title")
films_extern = films_extern.drop_duplicates(subset="title")

In [ ]:
# We halen een whitespace weg in de eerste dataset die er niet in hoort te zitten.
# We slicen de laatste 7 characters van de title column (het jaar dat de film uitkwam) in de tweede dataset weg,
# zodat de column overeen komt met de column in de eerste dataset.
films.update(films['movie_title'].str[:-1])
films_extern.update(films_extern['title'].str[:-7])

In [ ]:
# We setten de indexes van beide datasets naar de titel van de films.
films.set_index('movie_title', inplace=True)
films_extern.set_index('title', inplace=True)

In [ ]:
# We sorteren beide datasets zodat de waardes gelijk zullen zijn.
films = films.sort_index()
films_extern = films_extern.sort_index()

Nu ziet de eerste dataset er zo uit:

In [ ]:
films.head()

En ziet de tweede dataset er zo uit:

In [ ]:
films_extern.head()

Nu gaan we beide datasets samenvoegen tot 1 dataframe, deze noemen we 'films_extern_merge'. Het idee is om de youtubeId column van de tweede dataset te verwerken in de eerste dataset. Deze gemergde dataset gaan we later gebruiken.

In [ ]:
# Omdat de youtubeId column het enige is wat relevant is, is dat ook het enige veld dat we toevoegen. 
# De indexes van de eerste en tweede dataset worden samengevoegd tot 1 index genaamd title.
films_extern_merge = pd.merge(films, films_extern, left_index=True, right_index=True)
films_extern_merge.index.name = 'title'

# Tot slot droppen we de niet-relevante kollomen. Om code te besparen, overschrijven we alleen de dataframe met de relevante kollomen.
films_extern_merge = films_extern_merge[['gross','youtubeId']]
films_extern_merge

Je kunt zien dat er 2827 films zijn. Dit is bijna de helft minder dan er in de oorspronkelijke eerste dataset stond, dit is omdat we de onbruikbare data hebben verwijderd en er alleen bruikbare data is overgebleven.
We gaan de gemergede dataset nu verkennen en analyseren.

# Stap 4: Data exploration & analysis
---

Om de data te verkennen en er een betere grip op te krijgen zullen wij een paar commando's loslaten op de verwerkte data. Als eerste zullen wij een .describe gebruiken om een overzicht te krijgen van alle info van de numerieke waardes. Op deze manier kunnen wij bijvoorbeeld de gemiddelde lengte van een film zien. Dit is bij deze dataset dus 105 minuten.

In [ ]:
films.describe()

Om de data verder te verkennen en een beter gevoel te krijgen kunnen wij gebruik maken van grafieken. Zo staat hieronder bijvoorbeeld een staafdiagram van het aantal films per jaartal. In de x as staat het jaartal, deze zijn gesorteerd bij hoeveel films er in dat jaartal zijn uitgekomen die in deze dataset staan. De meeste films uit deze dataset komen dus uit 2002.

In [ ]:
films['title_year'].value_counts().plot(kind='bar', width=0.5,figsize=(20,5));
plt.xlabel("Jaartal", labelpad=14)
plt.ylabel("Aantal films", labelpad=14)
plt.title("Aantal films per jaartal", y=1.02);

## Onderzoeks vragen
---

Voor deze notebook moeten er verschillende onderzoeksvragen worden uitgewerkt, elk met hun eigen antwoord en andere manier van oplossen. Deze zullen nu 1 voor 1 worden uitgewerkt.

### Vraag 1: Hoeveel effect heeft de lengte van een trailer op de omzet van de film?
---

Voor deze vraag maken we gebruik van de gemergde dataset uit stap 3. Zoals je misschien al hebt kunnen raden bestaat de externe dataset uit ID's voor YouTube video's. Deze verwijzen naar de trailers van de films van de toegewezen dataset. We hebben bij stap 3 de toegewezen en externe dataset gemerged tot 1 dataset, deze gaan we nu gebruiken.

In [ ]:
films_extern_merge.iloc[:,-1].head()

Hier komt de pafy library ter zake. Om een voorbeeld te geven over wat het kan doen maken we eerst een video variabele aan:

In [ ]:
# Om het simpel te houden gebruiken we voor nu maar de youtubeId van de eerste film in de dataframe.
url = films_extern_merge.iloc[0, -1]
video = pafy.new(url)

Van deze video kunnen we nu verschillende data ophalen, hier een paar voorbeelden:

In [ ]:
# De titel van de video
print("De titel van de video is: \n" + str(video.title))
print("\n")

# De lengte van de video in secondes
print("De lengte van de video in secondes is: \n" + str(video.length))
print("\n")

# De concrete duratie van de video
print("De concrete duratie van de video is: \n" + str(video.duration))

Er zijn nog veel andere statistieken die kunnen worden opgehaad met behulp van pafy, maar hiervoor is een YouTube API account voor nodig en is ook niet voor ons relevant. We focussen ons op de <b>concrete duratie</b> van de video.

Dus, heeft de duratie van de trailer invloed op de omzet van de film? Om daar achter te komen maken we nieuwe dataframe aan, deze pakt 100 willekeurige films uit de dataset:

In [ ]:
# De dataframe heet "random_films".
random_films = films_extern_merge.sample(n = 100)
# In deze list slaan we zometeen de duraties op.
duratie = []
# We loopen door random_films heen en extracten van elke video de duratie in secondes.
# LET OP! Dit duurt een tijdje, aangezien er elke keer opnieuw een HTTP request wordt gemaakt. Je kunt ook de errors en warnings negeren.
i = 0
for x in random_films['youtubeId'].values:
    try:
        duratie.append(pafy.new(x).length)
        print(i)
        i = i+1
    except Exception:
        # Sommige video's zijn niet meer te zien op YouTube, hier kan de data dus ook niet van worden opgehaald.
        duratie.append(None)
        print(i)
        i = i+1
        continue

# We voegen een kolom toe met de waardes van duratie.
random_films['trailer_length'] = duratie
# We halen alle trailers eruit met een duratie van meer dan 400 seconden, aangezien deze vaak niet kloppen.
random_films = random_films[random_films.trailer_length < 400]
random_films

Nu maken we een mooie scatterplot om te zien of er een verband is tussen de omzet en trailer duratie:

In [ ]:
sns.regplot(x=random_films["trailer_length"], y=random_films["gross"])

Zoals je kunt zien komen films qua omzet veel met elkaar overeen, ongeacht de lengte van de trailers.
We kunnen dus concluderen dat er <b>geen</b> verband is.